Francisco Javier Piqueras Martínez

# TP3 - Apartado 2 - Infraestructuras Computacionales para el Procesamiento de Datos Masivos
## Ejercicio 3

Una de las aplicaciones más interesantes que se pueden realizar sobre la información que se recopila de Twitter es el denominado análisis de sentimiento o sentiment analysis. Se trata fundamentalmente de clasificar de forma automática la polaridad de un tweet en concreto, en función de su contenido, categorizándolo como positivo, negativo o neutro, e incluso asignándole un valor numérico a dicha clasificación. Esta funcionalidad se utiliza, por ejemplo, en sistemas de análisis de reputación online (Online Reputation Management, ORM) para determinar qué se está diciendo de una determinada marca en Twitter, en tiempo real.

En este ejercicio vamos a simular, de forma simplificada, un sistema de tiempo real que ofrezca una puntuación de polaridad (positiva, negativa o neutra) a los tweets que vayan llegando. Para ello, el modelo de clasificación se basará en listas de palabras, denominadas lexicones. En concreto, dispondremos de un lexicón de palabras positivas y su puntuación, “positive_lex.txt”, y de un lexicón de palabras negativas y su puntuación, “negative_lex.txt”. Ambos contienen, en cada línea, una palabra (positiva o negativa, respectivamente), seguida de su puntuación. Nótese que la polaridad negativa ya está expresada con un signo menos.

Se pide desarrollar un notebook de Jupyter, denominado “sentiment.ipynb”, en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter. Para cada nuevo tweet que llegue al sistema, se analizará su polaridad dividiendo el tweet por palabras y realizando la suma de la polaridad de todas las palabras que aparezcan en los ficheros “positive_lex.txt” y “negative_lex.txt”. Las palabras del lexicon negativo tienen un signo menos delante, por lo que restarán a la puntuación final del tweet. Si ninguna de las palabras del tweet está almacenada en los ficheros proporcionados (o si las puntuaciones de las palabras positivas y negativas se anulan), la polaridad del tweet será 0.

La salida del sistema ha de imprimirse cada segundo, y consistirá en una línea por cada tweet recibido, que contenga el texto del propio tweet, a continuación la puntuación de su polaridad, y la palabra “NEUTRO” si la puntuación es igual 0, “POSITIVO” si es mayor que 0, y “NEGATIVO” si es menor que 0.

In [1]:
# Leemos todas las palabras con connotación positiva del arcihvo positive_lex.txt
positive_words = []
with open('../resources/positive_lex.txt', 'r') as p:
    pos = [line.strip() for line in p]
for line in pos:
    positive_words.append(line.split(' '))

In [2]:
# Leemos todas las palabras con connotación negativa del archivo negative_lex.txt
negative_words = []
with open('../resources/negative_lex.txt', 'r') as n:
    neg = [line.strip() for line in n]
for line in neg:
    negative_words.append(line.split(' '))

Realizamos todos los imports necesarios:

In [3]:
import findspark
findspark.init('/Users/javierpiquerasmartinez/dev/spark-2.4.4-bin-hadoop2.7')

In [4]:
import os
os.environ['JAVA_HOME']='/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/jre'
os.environ['PYSPARK_SUBMIT_ARGS']='--master local[*] --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [5]:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

Inicializamos SparkContext y el StreamingContext, con una duración de batch de 1 segundo.

In [6]:
sc = SparkContext(master="local[*]", appName="Hashtags")
ssc = StreamingContext(sc, batchDuration=1)
ssc.checkpoint("checkpoint")

Para la recepción de datos, creamos un DirectStream que atienda a el evento 'kafkaTwitter' en el servidor localhost:9092.

In [7]:
dstream = KafkaUtils.createDirectStream(
    ssc, 
    topics = ['kafkaTwitter'],
    kafkaParams = {
        'bootstrap.servers' : 'localhost:9092'
    })

Definimos las funciones que nos van a ayudar a calcular la puntuación y el resultado.

In [8]:
def getScore(tweet):
    words = tweet.split(' ');
    sc = 0
    for pos_word in positive_words:
        for word in words:
            if(pos_word[0] == word):
                sc+=float(pos_word[1])
    for neg_word in negative_words:
        for word in words:
            if(neg_word[0] == word):
                sc+=float(neg_word[1])
    return sc

def getSentiment(score):
    if score>0:
        return 'POSITIVO'
    elif score<0:
        return 'NEGATIVO'
    else:
        return 'NEUTRO'

Formamos la respuesta para su devolución:

In [9]:
result = dstream.map(
    lambda tweet: [tweet[1], getScore(tweet[1]), getSentiment(getScore(tweet[1]))]
)

In [10]:
result.pprint()

In [11]:
ssc.start()

-------------------------------------------
Time: 2020-01-12 19:33:09
-------------------------------------------
['"Los problemas de la gente no se solucionan con escenografías, maquillajes o cambios de logos" @pbustinduy #L6Nbarómetro', 0.375, 'POSITIVO']
['@VideojuegosGAME gracias :D', 0, 'NEUTRO']
['Dedicado a los que inician parciales...porque yo tambn pasé por ahí @JUANCAORTIZ14 @los40colombia #Noches40 :D http://t.co/c9m35mCBtR', 0, 'NEUTRO']
["Los globos de Google 'con internet para todos' ya están casi listos (video) - http://t.co/jQp8xEub0V http://t.co/GFdIEkwF0B", 0, 'NEUTRO']
['Nena, hiciste que mi varita se levantara sin usar Wingardium Leviosa. ..#FrasesparaligardeHoward', 0, 'NEUTRO']
['Comienza la intervención de @marianorajoy. Ya en directo en http://t.co/08eZ0fzjAi #TrabajarHacerCrecer', 0.26, 'POSITIVO']
['@TownGamePlay feliz domingo, mañana entró de vacaciones :D', 1.0939999999999999, 'POSITIVO']
['Buenas tardes gentecilla!! :D.Tremenda viciadita me he pegado hoy al

-------------------------------------------
Time: 2020-01-12 19:33:16
-------------------------------------------
['@GarciaG_Andres Si no hay lista de espera será pq se han cubierto las plazas. De todas formas, ponte en contacto con @UAM_Medicina Saludos', 0.625, 'POSITIVO']
['@Ana_Libroblogs ¡Sí, pero hace muchísimo tiempo! Recuerdo que tenía un mundo muy interesante y Yoko como protagonista era genial :D', 0.849, 'POSITIVO']
['Así es Endgame: la novela que se convierte en realidad gracias a Ingress http://t.co/0oIX8EZruz http://t.co/A4k6uIv8v6', 0, 'NEUTRO']
['@MartellitasOffi Hola :D ¿De casualidad sabes que precio tienen? :33', 0, 'NEUTRO']
['No nací para estar de novia. Me estresa demasiado los gatos y los pelotudos que les siguen el juego bai', 0.375, 'POSITIVO']
['@nazary_gr  Emi, Ceballos, Eraso y nauzet/pina, juego con 4 medios. Venderías a Nauzet/pina por Sarabia? Tendría dos del Getafe..', 0, 'NEUTRO']
['Titanic versión pingüino.  http://t.co/m70rqXQJcw', 0, 'NEUTRO']
['Segu

-------------------------------------------
Time: 2020-01-12 19:33:23
-------------------------------------------
['Agradezco su labor al @ppvasco, sinónimo de libertad, lucha contra el terrorismo y apoyo constante a las víctimas #TrabajarHacerCrecer', 0.21899999999999997, 'POSITIVO']
['Ya disponible #APP de cultura científica #UAM @UAM_Gazette ,\u200ben Apple Store y Google Play http://t.co/drCapI5syB http://t.co/lODzHFk5zQ', 0.292, 'POSITIVO']
['Que tal como va todo!? Le acabo de contar al taxista como me gano la vida como hacker informatico😂 le he dicho que por favor no me delate!', 1.25, 'POSITIVO']
['Seguimos jugando a Dying Light. ¡En cooperativo! http://t.co/DVgKmNTu5u', 0, 'NEUTRO']
['Lo mejor y lo peor del... Samsung Galaxy S5 - http://t.co/VSmruXJGBQ http://t.co/1AdtmRAdOP', 0.375, 'POSITIVO']
['@SaraRamirez Tuitea más seguido en español, me hace feliz que lo hagas :D', 0.719, 'POSITIVO']
['@WillyrexYT  @YouTube te queremos =) sos lo mejor que hay en el planeta #APOCALIPSISMI

-------------------------------------------
Time: 2020-01-12 19:33:30
-------------------------------------------
['@eduardogui84 Hola :D aquí te comparto algo de música nueva http://t.co/GAv4Nwt4wB  Espero te agrade http://t.co/5nf0GrVP3C  Saludos^^', 0, 'NEUTRO']
['Ay qué día tan estupendo :D', 0, 'NEUTRO']
['@Arnau_xPop @Burning_Light_ Me recomendarian algun juego para pc? De pokemon ^^', 0, 'NEUTRO']
['@anxietyblow Takero mucho :D jajaja xD', 0, 'NEUTRO']
['.@PPrada2015 ”El PP en el Exterior aportará nuevas propuestas al Programa Electoral” http://t.co/CHaE6CM4Qr', 0.25, 'POSITIVO']
['"La conferencia ha servido para hacer terapia, para reflexionar y ver qué tenemos que cambiar, hacer y explicar" @ccifuentes #Cifuentes13tv', 0, 'NEUTRO']
['Matt Groening se quedo sin extras para este episodio… http://t.co/jb6X5pktuf', 0, 'NEUTRO']
['El robot albañil Hadrian podría colocar los ladrillos de una casa completa en un par de días https://t.co/rOWiiI8ciG http://t.co/On2Wvt96SK', -0.375, 'NE